# Extract Transform Load
In this notebook, we extract data from the praise worksheet, transform it into suitable, clean data, and load it into the outputs directory. 

In [1]:
import pandas as pd
import numpy as np
import re

## Total Hours Dataset

In [2]:
total_hours = pd.read_excel("data/TEC Praise Quantification.xlsx", sheet_name="Total Impact Hours so far", engine='openpyxl', header=1, index_col=0, usecols="A:D")
total_hours = total_hours.reset_index().dropna(thresh=2).reset_index()[['Handle', 'Impact Hours']]
total_hours.to_csv('outputs/total_hours.csv', index=False)
total_hours.head(15)

,Handle,Impact Hours
0,zeptimusq,686.601440
1,santigs67,621.789072
2,ygganderson,502.691508
3,cranders71,418.671961
4,sembrestels,380.123299
5,solsista,353.034515
6,akrtws,332.309293
7,mzargham,289.446628
8,iviangita,265.214736
9,naynaysoo,251.147051


## Praise Dataset
We consider three batches of praise. Batch one is October 2020, and December2020-May2021. Batch two is prior to December 2020 (other than October). Batch three is after May 7th 2021. Batch one is the original batch that all analysis was performed on. Chronologically, prior to Batch one (batch two) is the early days of praise, and batch 3 is praise that started after the initial praise analysis began.

#### Batch One

In [3]:
periods = [
    "#17 May 7",
    "#16 Apr 24",
    "#15 Apr 9",
    "#14 Mar 26",
    "#13 Mar 12",
    "#12 Feb 26",
    "#11 Feb 12",
    "#10 Jan 29",
    "#9 Jan 15", 
    "#8 Jan 1",
    "#7 Dec 18",
    "#6 Dec 4",
    "#2 Oct 9",

]

In [4]:
def read_period_from_sheet(period: str, batch: str) -> pd.DataFrame:
        # Load the data
        df = pd.read_excel('data/TEC Praise Quantification.xlsx', skiprows=2, sheet_name=period,engine='openpyxl', usecols="A:M")
        
        # Add a period column
        df['period'] = period
        df['batch'] = batch
        
        # Remove the validator normalization as it is confusing and unecessary for analysis
        df.columns = list(df.columns[:6]) + ['v1 norm', 'v2 norm', 'v3 norm'] + list(df.columns[9:])
        df = df.dropna(thresh=8).drop(['v1 norm', 'v2 norm', 'v3 norm'], axis=1)
        
        # Return the loaded df
        return df

In [5]:
# Load and concatenate data
data = pd.concat([read_period_from_sheet(p, batch='Batch 1') for p in periods])

In [6]:
# Combine these duplicate columns and drop the lesser named one.
df = data.copy()
# To
df['To'] = df['To'].combine_first(df['To.1']).combine_first(df['Unnamed: 12'])
df = df.drop(['To.1', 'Unnamed: 12'], axis=1)

# IH Per Praise
df['IH per Praise'] = df['IH per Praise'].combine_first(df['Cred per Praise'])
df = df.drop('Cred per Praise', axis=1)

# IH Per Person Per Period
df['IH per person'] = df['IH per person'].combine_first(df['Cred per person'])
df = df.drop('Cred per person', axis=1)

# Rename The Institution Column
df = df.rename({'Unnamed: 3':'Institution'}, axis=1)

In [7]:
data.groupby('period').count()

,To,From,Reason for dishing,Unnamed: 3,Date,Room,Avg %,IH per Praise,IH per person,Unnamed: 12,batch,Cred per Praise,Cred per person,To.1
period,,,,,,,,,,,,,,
#10 Jan 29,575,575,575,552,552,552,573,569,575,575,575,0,0,0
#11 Feb 12,463,463,463,438,438,438,463,454,463,422,463,0,0,0
#12 Feb 26,672,672,672,646,646,646,662,657,672,672,672,0,0,0
#13 Mar 12,651,651,651,624,624,624,651,649,651,651,651,0,0,0
#14 Mar 26,804,804,804,774,774,774,804,797,804,804,804,0,0,0
#15 Apr 9,963,963,963,932,932,932,963,963,963,963,963,0,0,0
#16 Apr 24,908,908,908,875,875,875,908,905,908,908,908,0,0,0
#17 May 7,974,974,974,940,940,940,974,971,974,974,974,0,0,0
#2 Oct 9,124,124,124,124,124,124,124,0,0,0,124,124,124,124


#### Batch Two

In [8]:
df.head(1)

,To,From,Reason for dishing,Institution,Date,Room,Avg %,IH per Praise,IH per person,period,batch
0,zeptimusQ,Tam2140#9361,for hosting this kicking params party!,Token Engineering Commons,May-07-2021,🙏praise,0.001963,2.355252,39.04072,#17 May 7,Batch 1


In [9]:
periods = [
    "#5 Nov 20", 
    "#4 Nov 6", 
    "#3 Oct 23", 
    "#1 Sept 24", 
    "#0 Sept 7 (historic)", 
]

In [10]:
data2 = pd.concat([read_period_from_sheet(p, batch='Batch 2') for p in periods])

In [11]:
# Combine these duplicate columns and drop the lesser named one.
df2 = data2.copy()
# To
df2['To'] = df2['To'].combine_first(df2['To.1'])
df2 = df2.drop(['To.1'], axis=1)

# IH Per Praise
df2 = df2.rename({'Cred per Praise':'IH per Praise'}, axis=1)

# IH Per Person Per Period
df2 = df2.rename({'Cred per person':'IH per person'}, axis=1)

# Rename The Institution Column
df2 = df2.rename({'Unnamed: 3':'Institution'}, axis=1)

#### Batch 3

In [12]:
periods = [
    "#18 May 21",
    "#19 Jun 4",
    "#20 Jun 18",
    "#21 July 1",
    "#22 July 11",
]

In [13]:
data3 = pd.concat([read_period_from_sheet(p, batch='Batch 3') for p in periods])

In [14]:
# Combine these duplicate columns and drop the lesser named one.
df3 = data3.copy()
# To
df3['To'] = df3['To'].combine_first(df3['Unnamed: 12'])
df3 = df3.drop(['Unnamed: 12'], axis=1)

# Rename The Institution Column
df3 = df3.rename({'Unnamed: 3':'Institution'}, axis=1)

### Concatenate Praise Batches into Praise Dataset

In [15]:
data = pd.concat([df, df2, df3])

### Resolving Names

In [16]:
names_df = pd.read_excel('data/TEC Praise Quantification.xlsx',sheet_name="DO NOT TOUCH Imported",engine='openpyxl', usecols="A:D")

#### We create a dictionary that matches each non-null entry in the spreadsheet to its "IH & CSTK Handle"
source_cols = names_df.columns
names_dict = {} #create a blank dictionary

for i in range(len(names_df)):
    for col in source_cols:
        name_to_consider = names_df.loc[i, col]
        canonical_name = names_df.loc[i,"IH & CSTK Handle"]
        if pd.isna(canonical_name):
            canonical_name = name_to_consider
        if not(pd.isna(name_to_consider)):
            names_dict[name_to_consider] = canonical_name

#### Did we catch them all? See if there's anything in the combned user sets of "To" and "From" that isn't a key in the names_dict dictionary. 
combined_users = set([]).union(set(data["From"]),set(data["To"]))
names_uncaught = sorted([str(x) for x in combined_users if (not(x in names_dict.keys()) and not(pd.isna(x)))])

# print("All told, there are {} names in our data set that lack canonical representations: \n".format(str(len(names_uncaught))))
# print("We do not have canonical representations for the following names: \n")
# for name in names_uncaught:
#     print(str(name))

def clean_name(name):
    new_name = str(name)
    new_name = new_name.lower()
    new_name = re.sub('#\d\d\d\d','',new_name)
    new_name = re.sub('[^A-Za-z0-9]+', '', new_name) #remove all non-alphanumeric characters
    return new_name

clean_name("zeptimusQ")
clean_name("ygg_anderson")
clean_name("AmwFund#0979")
clean_name("Caeser (PST)#0046")

cleaned_keys = []
original_keys = list(names_dict.keys())
for key in original_keys:
    clean_key = clean_name(key)
    cleaned_keys.append(clean_key)
    if not(clean_key == key):
        names_dict[clean_key] = names_dict[key]
    
new_pairs = {}

for name in names_uncaught:
    clean_version = clean_name(name) #clean the name 
    key_matches = [key for key in cleaned_keys if clean_version == key] #make a list of all keys with same cleaned name
    if len(key_matches) > 0:
        for match in key_matches:
            if match in names_dict.keys():
                right_name = names_dict[match]
                new_pairs[name] = right_name
                names_dict[name] = right_name
                break
                    
# print("We were able to add the following names: \n")
# for name in new_pairs.keys():
#     print(str(name) + "\t ----> \t" + str(names_dict[name]))

still_uncaught_names = sorted([str(x) for x in combined_users if (not(x in names_dict.keys()))])
num_still_uncaught_names = str(len(still_uncaught_names))
print("We are still missing {} names".format(num_still_uncaught_names))

We are still missing 10 names


In [17]:
def total_appearances(name, data):
    from_appearances = sum(data["From"] == name)
    to_appearances = sum(data["To"] == name)
    total = from_appearances + to_appearances
    return total 

still_uncaught_appearances = [total_appearances(x, data) for x in still_uncaught_names]

uncaught_df = pd.DataFrame({"name": still_uncaught_names, "appearances" : still_uncaught_appearances})

for name in still_uncaught_names:
    names_dict[name] = name

last_remaining = [user for user in combined_users if(not(user in names_dict.keys()))]
num_remaining = len(last_remaining)
print("There are {} remaining users with no representation.".format(num_remaining))
print("These users are \n")

for name in last_remaining:
    print(name)


There are 0 remaining users with no representation.
These users are 



In [18]:
import pickle

In [19]:
with open('outputs/names_dict.pickle', 'wb') as f:
    pickle.dump(names_dict, f)

Map the names in the dataframe to the resolved names.

In [20]:
data.loc[:,"To"] = data.loc[:,"To"].map(names_dict)
data.loc[:, "From"] = data.loc[:,"From"].map(names_dict)

### Save Cleaned Praise Dataset

In [21]:
data.to_csv('outputs/praise_data.csv',index=False)

### Split Receivers and Quantifiers

In [22]:
receivers = data[(~data[['Institution', 'Date', 'Room']].isna().all(axis=1)) & (data['From'] != 'Quantifiers')]
quantifiers = data[(data[['Institution', 'Date', 'Room']].isna().all(axis=1)) | (data['From'] == 'Quantifiers')]
len(receivers) + len(quantifiers) == len(data)

True

In [23]:
receivers.to_csv('outputs/receivers.csv', index=False)
quantifiers.to_csv('outputs/quantifiers.csv', index=False)

In [24]:
data['period'].unique()

array(['#17 May 7', '#16 Apr 24', '#15 Apr 9', '#14 Mar 26', '#13 Mar 12',
       '#12 Feb 26', '#11 Feb 12', '#10 Jan 29', '#9 Jan 15', '#8 Jan 1',
       '#7 Dec 18', '#6 Dec 4', '#2 Oct 9', '#5 Nov 20', '#4 Nov 6',
       '#3 Oct 23', '#1 Sept 24', '#0 Sept 7 (historic)', '#18 May 21',
       '#19 Jun 4', '#20 Jun 18', '#21 July 1', '#22 July 11'],
      dtype=object)

In [25]:
data

,To,From,Reason for dishing,Institution,Date,Room,Avg %,IH per Praise,IH per person,period,batch
0,zeptimusQ,Tam2140,for hosting this kicking params party!,Token Engineering Commons,May-07-2021,🙏praise,0.001963,2.355252,39.040720,#17 May 7,Batch 1
1,zeptimusQ,iviangita,for hosting and leading a lot of params parties,Token Engineering Commons,May-07-2021,🙏praise,0.001663,1.995539,39.040720,#17 May 7,Batch 1
2,zeptimusQ,JuankBell,for testing and deploying the bot to record an...,Token Engineering Commons,Apr-28-2021,🙏praise,0.001342,1.610872,39.040720,#17 May 7,Batch 1
3,zeptimusQ,iviangita,for the huge success of the MVV process,Token Engineering Commons,Apr-30-2021,🙏praise,0.001043,1.251160,39.040720,#17 May 7,Batch 1
4,zeptimusQ,iviangita,"for his awesome work on the recorder bot, for ...",Token Engineering Commons,Apr-30-2021,🙏praise,0.001043,1.251160,39.040720,#17 May 7,Batch 1
...,...,...,...,...,...,...,...,...,...,...,...
1129,andythegreenie#7463,iviangita,for mentioning or retweeting Commons Stack on ...,Commons Stack,2021-07-02 00:00:00,🙌praise,0.000205,0.410317,0.410317,#22 July 11,Batch 3
1130,kei#9866,iviangita,@David (please DYOR...) @KZ Flyer for mentioni...,Commons Stack,2021-07-11 00:00:00,🙌praise,0.000205,0.410317,0.410317,#22 July 11,Batch 3
1131,lefterisjp,iviangita,@lefterisjp | Rotki for mentioning or retweeti...,Commons Stack,2021-07-11 00:00:00,🙌praise,0.000205,0.410317,0.410317,#22 July 11,Batch 3
1132,fabiosmendes,Quantifiers,Gets paid from General Magic so he gets 50% of...,NaN,NaN,NaN,0.000000,0.000000,0.000000,#22 July 11,Batch 3


# Audit Corrections

In [28]:
results = pd.read_excel("data/TEC Praise Quantification.xlsx", sheet_name="Deduction Audit Results", engine='openpyxl', header=1, usecols="A:D")
results.to_csv('outputs/audit_results.csv',index=False)

In [29]:
results

,Handle,Impact Hours Calc,Impact Hours Calc.1,Why?
0,naynaysoo,125.868000,125.868000,Was deducted but not paid for several months
1,freedumbs00,73.345850,73.345850,Was deducted but not paid for several months
2,fabiosmendes,10.639598,10.639598,Was deducted more than he should have been
3,fabimol,9.260812,9.260812,Was deducted but not paid for several months
4,geleeroyale,5.867313,5.867313,Was deducted but not paid for several months
5,jeffemmett,4.397127,4.397127,Was deducted more than he should have been
6,Gfriis,3.100002,3.100002,Was deducted more than he should have been
7,vivszaid,1.683976,1.683976,Was deducted more than she should have been
8,vegayp,1.470148,1.470148,Was deducted more than he should have been
9,iviangita,1.368500,1.368500,Was deducted more than she should have been
